# Final Capstone: Revisiting the Netflix Prize

## Notebook 2: Feature Engineering

With few given features to work with, and a $1,000,000 reward up for grabs, contest participants looked for ways to extract information in the data that could be used explicitly to better represent user preferences and bias, and somehow relate them to movie attributes. Furthermore, inclusion of the rating date adds a time dimension to these somewhat implicit relationships. The goal then, is to find relationships that may improve prediction accuracy, determine the appropriate numerical calculation, code the operation, then use the output as a new feature in the dataset.

Tasks such as these are creative in nature; but working with big data requires the data science practitioner to always be aware of the state of computational resources. Perhaps the most challenging aspect, however, is choosing the most efficient path in processing the data.

In [1]:
import time
start_time = time.perf_counter()
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
%%time
# retrieve data exported from first notebook
base_path = 'C:/Users/jnpol/Documents/DS/Data Science/UL/'
all_ratings = pd.read_parquet(base_path + 'all_ratings.parquet')
quindex = pd.read_parquet(base_path + 'quindex.parquet')
net = pd.read_parquet(base_path + 'net1.parquet')

all_ratings.info()
print()
quindex.info()
print()
net.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 0 to 100480506
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   rating  int8 
dtypes: int8(1)
memory usage: 838.7 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408395 entries, 0 to 1408394
Data columns (total 1 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   quindex  1408395 non-null  int64
dtypes: int64(1)
memory usage: 10.7 MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 0 to 100480506
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   mov_id     int16  
 1   cust_id    int32  
 2   rating     float64
 3   day_rated  int16  
 4   mov_year   int16  
dtypes: float64(1), int16(3), int32(1)
memory usage: 2.4 GB
Wall time: 3.32 s


## New Features: Counts
The features below are intended to reflect the patterns, and tendencies, behaviors (and temporal shifts) of the movie connoisseur, as well as express movie attributes. Some features are designed to provide more clarity on general trends, while others seek to illuminate specific traits of each customer.

In [3]:
%%time
# add column indicating number of times movie was rated
net['mov_count'] = net.groupby(['mov_id'])['mov_id'].transform('count')

# add column indicating average number of ratings per movie per day
net['avg_rate_pm_pd'] = net.mov_count / net.day_rated.max()
net.drop(['mov_count'], 1, inplace=True)

# add column indicating the number of movies rated per cust
net['rated_bycust'] = net.groupby(['cust_id'])['cust_id'].transform('count')

# add column indicating average number of ratings per customer per day
net['avg_rate_pc_pd'] = net.rated_bycust / net.day_rated.max()
net.drop(['rated_bycust'], 1, inplace=True)

# add column indicating number of times cust rated on that day
net['cust_day_count'] = net.groupby(
    ['cust_id', 'day_rated'])['mov_id'].transform('count')
net.cust_day_count = net.cust_day_count.astype(np.int16)

# add column indicating number of days since customer's first rating
net['day_min'] = net.groupby(['cust_id'])['day_rated'].transform('min')
net['cust_days_since'] = net.day_rated - net.day_min
net.drop(['day_min'], 1, inplace=True)

# add column indicating number of days since movie's first rating
net['day_min'] = net.groupby(['mov_id'])['day_rated'].transform('min')
net['mov_days_since'] = net.day_rated - net.day_min
net.drop(['day_min'], 1, inplace=True)

Wall time: 49.4 s


## Additional Features: Means
The additional features below may be calculated on the training set only. They cannot be applied directly on the quiz set since the ratings are assumed to be unknown; but they can (and will) be estimated. In this way, even though the quiz set's true ratings cannot be use to train the model, such features that calculate averages on ratings from the training data can be added to the quiz set.

Note that there are many opportunities to reduce memory consumption. This continues throughout the project.

In [4]:
%%time
# used to select rows matching original quiz df index
quilist = list(quindex.quindex)

# add column indicating average rating per movie
net['mov_avg_rating'] = net.drop(quilist).groupby(
    ['mov_id'])['rating'].transform('mean')
net.mov_avg_rating = net.mov_avg_rating.astype(np.float32)

# add column indicating average rating per cust
net['cust_avg_rating'] = net.drop(quilist).groupby(
    ['cust_id'])['rating'].transform('mean')
net.cust_avg_rating = net.cust_avg_rating.astype(np.float32)

# add column indicating average rating per per day
net['mov_day_avg'] = net.drop(quilist).groupby(
    ['mov_id', 'day_rated'])['rating'].transform('mean')
net.mov_day_avg = net.mov_day_avg.astype(np.float32)

# add column indicating daily average rating by the cust
net['cust_day_avg'] = net.drop(quilist).groupby(
    ['cust_id', 'day_rated'])['rating'].transform('mean')
net.cust_day_avg = net.cust_day_avg.astype(np.float32)

# add column indicating the difference between cust rating and mov_avg_rating
net['cust_rating_offset'] = net.rating - net.mov_avg_rating
net.cust_rating_offset = net.cust_rating_offset.astype(np.float32)

# add column indicating avg offset per customer
net['cust_avg_offset'] = net.drop(quilist).groupby(
    ['cust_id'])['cust_rating_offset'].transform('mean')
net.cust_avg_offset = net.cust_avg_offset.astype(np.float32)

Wall time: 2min 46s


In [5]:
%%time
# add column indicating average rating per release year
net['avg_rate_yr'] = net.drop(quilist).groupby(
    ['mov_year'])['rating'].transform('mean')
net.avg_rate_yr = net.avg_rate_yr.astype(np.float32)

# add column indicating average rating per release year per day
net['avg_rl_day'] = net.drop(quilist).groupby(
    ['mov_id', 'mov_year', 'day_rated'])['rating'].transform('mean')
net.avg_rl_day = net.avg_rl_day.astype(np.float32)

# add column indicating average rating per customer per release year
net['avg_rate_cst_yr'] = net.drop(quilist).groupby(
    ['cust_id', 'mov_year'])['rating'].transform('mean')
net.avg_rate_cst_yr = net.avg_rate_cst_yr.astype(np.float32)

Wall time: 1min 50s


The code below shows the quantity of null values in each column. Most of these values will be filled shortly; but not all of them. We will see that this allows for a convenient method of row selection.

In [6]:
%%time
net.info()
display(net.head())
net.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 0 to 100480506
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   mov_id              int16  
 1   cust_id             int32  
 2   rating              float64
 3   day_rated           int16  
 4   mov_year            int16  
 5   avg_rate_pm_pd      float64
 6   avg_rate_pc_pd      float64
 7   cust_day_count      int16  
 8   cust_days_since     int16  
 9   mov_days_since      int16  
 10  mov_avg_rating      float32
 11  cust_avg_rating     float32
 12  mov_day_avg         float32
 13  cust_day_avg        float32
 14  cust_rating_offset  float32
 15  cust_avg_offset     float32
 16  avg_rate_yr         float32
 17  avg_rl_day          float32
 18  avg_rate_cst_yr     float32
dtypes: float32(9), float64(3), int16(6), int32(1)
memory usage: 10.1 GB


,mov_id,cust_id,rating,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,cust_rating_offset,cust_avg_offset,avg_rate_yr,avg_rl_day,avg_rate_cst_yr
0,1,1488844,3.00,2125,2003,0.24,0.98,4,200,590,3.72,3.26,4.00,3.25,-0.72,-0.26,3.51,4.00,3.22
1,1,822109,5.00,2009,2003,0.24,0.07,11,36,474,3.72,3.99,5.00,4.36,1.28,0.41,3.51,5.00,4.00
2,1,885013,4.00,2168,2003,0.24,0.16,3,157,633,3.72,3.84,4.00,4.00,0.28,0.07,3.51,4.00,3.47
3,1,30878,nan,2236,2003,0.24,0.58,7,1462,701,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1,823519,3.00,1636,2003,0.24,0.29,34,41,101,3.72,3.90,3.00,3.91,-0.72,0.32,3.51,3.00,3.96


Wall time: 3.18 s


mov_id                      0
cust_id                     0
rating                1408395
day_rated                   0
mov_year                    0
avg_rate_pm_pd              0
avg_rate_pc_pd              0
cust_day_count              0
cust_days_since             0
mov_days_since              0
mov_avg_rating        1408395
cust_avg_rating       1408395
mov_day_avg           1408395
cust_day_avg          1408395
cust_rating_offset    1408395
cust_avg_offset       1408395
avg_rate_yr           1408395
avg_rl_day            1408395
avg_rate_cst_yr       1408395
dtype: int64

Before filling in missing values, the dataframe is sorted so that the values designated to fill in the nulls are positioned in the previous row. All columns except for 'rating' are forward filled. For each feature, the dataframe is re-sorted. Notice that the first 4 features all begin with 'cust_id' as the primary value to sort by. This is intentional, and helps expedite the task.

In [23]:
# %%time
# temp_df = net[net.isna().any(axis=1)].copy()
# temp_df.cust_rating_offset = temp_df.cust_avg_offset
# net = net.merge(temp_df, on= , right_index=True)

Wall time: 7.39 s


In [6]:
net.drop(['cust_rating_offset'], 1, inplace=True)

In [7]:
%%time
net.sort_values(by=['cust_id', 'mov_year', 'avg_rate_cst_yr'], inplace=True)
net.avg_rate_cst_yr.fillna(method='ffill', inplace=True)

net.sort_values(by=['cust_id', 'mov_year', 'avg_rate_cst_yr'], inplace=True)
net.avg_rate_cst_yr.fillna(method='ffill', inplace=True)

net.sort_values(by=['cust_id', 'cust_avg_rating'], inplace=True)
net.cust_avg_rating.fillna(method='ffill', inplace=True)

net.sort_values(by=['cust_id', 'cust_avg_offset'], inplace=True)
net.cust_avg_offset.fillna(method='ffill', inplace=True)

net.sort_values(by=['cust_id', 'day_rated', 'cust_day_avg'], inplace=True)
net.cust_day_avg.fillna(method='ffill', inplace=True)

Wall time: 2min 13s


In [8]:
%%time
net.sort_values(by=['mov_id', 'mov_year', 'day_rated', 'avg_rl_day'], inplace=True)
net.avg_rl_day.fillna(method='ffill', inplace=True)

net.sort_values(by=['mov_id', 'day_rated', 'mov_day_avg'], inplace=True)
net.mov_day_avg.fillna(method='ffill', inplace=True)

net.sort_values(by=['mov_id', 'mov_avg_rating'], inplace=True)
net.mov_avg_rating.fillna(method='ffill', inplace=True)

net.sort_values(by=['mov_year', 'avg_rate_yr'], inplace=True)
net.avg_rate_yr.fillna(method='ffill', inplace=True)

Wall time: 1min 47s


In [9]:
%%time
# add column for difference between global mean and cust mean
net['cust_glob_diff'] = net.rating.mean() - net.cust_avg_rating

# add column for difference between global mean and mov mean
net['mov_glob_diff'] = net.rating.mean() - net.mov_avg_rating

# manage memory
net.cust_glob_diff = net.cust_glob_diff.astype(np.float32)
net.mov_glob_diff = net.mov_glob_diff.astype(np.float32)

Wall time: 1.07 s


In [10]:
%%time
net.info()
display(net.head())
net.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 43515483 to 100349765
Data columns (total 21 columns):
 #   Column              Dtype  
---  ------              -----  
 0   mov_id              int16  
 1   cust_id             int32  
 2   rating              float64
 3   day_rated           int16  
 4   mov_year            int16  
 5   avg_rate_pm_pd      float64
 6   avg_rate_pc_pd      float64
 7   cust_day_count      int16  
 8   cust_days_since     int16  
 9   mov_days_since      int16  
 10  mov_avg_rating      float32
 11  cust_avg_rating     float32
 12  mov_day_avg         float32
 13  cust_day_avg        float32
 14  cust_rating_offset  float32
 15  cust_avg_offset     float32
 16  avg_rate_yr         float32
 17  avg_rl_day          float32
 18  avg_rate_cst_yr     float32
 19  cust_glob_diff      float32
 20  mov_glob_diff       float32
dtypes: float32(11), float64(3), int16(6), int32(1)
memory usage: 8.4 GB


,mov_id,cust_id,rating,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,cust_rating_offset,cust_avg_offset,avg_rate_yr,avg_rl_day,avg_rate_cst_yr,cust_glob_diff,mov_glob_diff
43515483,7654,1312412,3.00,116,1896,0.07,0.88,268,44,0,3.65,3.46,3.00,3.21,-0.65,-0.06,3.65,3.00,3.00,0.15,-0.04
43515504,7654,947104,3.00,254,1896,0.07,0.72,79,3,138,3.65,3.22,3.00,3.61,-0.65,-0.31,3.65,3.00,3.00,0.39,-0.04
43515485,7654,1830265,5.00,338,1896,0.07,0.73,242,0,222,3.65,3.42,5.00,3.43,1.35,-0.13,3.65,5.00,5.00,0.18,-0.04
43515426,7654,1328708,5.00,460,1896,0.07,0.36,215,0,344,3.65,3.34,5.00,3.40,1.35,-0.24,3.65,5.00,5.00,0.26,-0.04
43515455,7654,2449001,4.00,503,1896,0.07,0.63,5,70,387,3.65,3.35,4.00,4.00,0.35,-0.27,3.65,4.00,4.00,0.26,-0.04


Wall time: 3.17 s


mov_id                      0
cust_id                     0
rating                1408395
day_rated                   0
mov_year                    0
avg_rate_pm_pd              0
avg_rate_pc_pd              0
cust_day_count              0
cust_days_since             0
mov_days_since              0
mov_avg_rating              0
cust_avg_rating             0
mov_day_avg                 0
cust_day_avg                0
cust_rating_offset    1408395
cust_avg_offset             0
avg_rate_yr                 0
avg_rl_day                  0
avg_rate_cst_yr             0
cust_glob_diff              0
mov_glob_diff               0
dtype: int64

In [10]:
%%time
net['all_ratings'] = all_ratings.rating
net = net.sample(frac=1, random_state=171)
net.info()
net.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97713135 entries, 85479082 to 47559793
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   mov_id           int16  
 1   cust_id          int32  
 2   rating           float64
 3   day_rated        int16  
 4   mov_year         int16  
 5   avg_rate_pm_pd   float64
 6   avg_rate_pc_pd   float64
 7   cust_day_count   int16  
 8   cust_days_since  int16  
 9   mov_days_since   int16  
 10  mov_avg_rating   float32
 11  cust_avg_rating  float32
 12  mov_day_avg      float32
 13  cust_day_avg     float32
 14  cust_avg_offset  float32
 15  avg_rate_yr      float32
 16  avg_rl_day       float32
 17  avg_rate_cst_yr  float32
 18  cust_glob_diff   float32
 19  mov_glob_diff    float32
 20  all_ratings      int8   
dtypes: float32(10), float64(3), int16(6), int32(1), int8(1)
memory usage: 8.1 GB
Wall time: 1min 8s


,mov_id,cust_id,rating,day_rated,mov_year,avg_rate_pm_pd,avg_rate_pc_pd,cust_day_count,cust_days_since,mov_days_since,mov_avg_rating,cust_avg_rating,mov_day_avg,cust_day_avg,cust_avg_offset,avg_rate_yr,avg_rl_day,avg_rate_cst_yr,cust_glob_diff,mov_glob_diff,all_ratings
85479082,15157,2332582,1.00,1911,2000,0.25,0.19,1,402,509,2.88,3.08,1.00,1.00,-0.39,3.54,1.00,3.00,0.52,0.73,1
87948019,15627,2554257,5.00,2014,1981,4.64,0.17,12,385,1970,3.73,4.06,3.79,4.17,0.43,3.74,3.79,4.50,-0.46,-0.12,5
29057292,5320,2573731,5.00,2135,2005,15.48,0.03,1,51,281,3.30,4.55,3.33,5.00,1.06,3.55,3.33,4.80,-0.94,0.31,5
45036525,8011,1117152,5.00,2232,2002,3.38,1.31,32,814,1181,3.60,4.55,3.00,5.00,1.19,3.50,3.00,4.78,-0.95,0.01,5
60553373,11089,1146333,5.00,2179,2001,56.57,0.08,35,63,1600,4.28,3.72,4.27,4.06,-0.16,3.55,4.27,3.75,-0.12,-0.68,5


In [11]:
%%time
net.to_parquet('net2.parquet')

Wall time: 33.5 s


In [13]:
end_time = time.perf_counter()
prog_ex_time = end_time - start_time
print('This program executes in {} seconds.'.format(prog_ex_time))

This program executes in 700.1010670000001 seconds.


That's 10 minutes and 40 seconds.

In [3]:
# %%time
# net.sort_values(by=['cust_id', 'day_rated'], inplace=True)